# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
# %pip install egobox

We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = [[0., 25.]]
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = [[0., 3.], [0., 4.]]
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [ ]:
# Unconstrained 1D function
egor = egx.Egor(xspecs_xsinx)   # see help(egor) for options
res = egor.minimize(xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 0, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, gp: GpConfig { regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, recombination: Hard, theta_tuning: Full { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }, n_start: 10, max_eval: 50 }, infill_criterion: WB2, infill_optimizer: Cobyla, target: -inf, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Float(0.0, 25.0)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: MeanConstraint }
INFO:egobox

Optimization f=[-15.12510324] at [18.93520941]


In [7]:
# Constrained 2D function
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                trego=False,
            )   

In [8]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 10, n_cstr: 2, cstr_tol: Some([0.001, 0.001], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, gp: GpConfig { regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, recombination: Hard, theta_tuning: Full { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }, n_start: 10, max_eval: 50 }, infill_criterion: WB2, infill_optimizer: Cobyla, target: -5.5, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Float(0.0, 3.0), Float(0.0, 4.0)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy:

Optimization f=[-5.50818847e+00  5.33903737e-05  2.24372753e-04] at [2.32953349 3.17865498]
Optimization history: 
Inputs = [[0.43178817 0.42424357]
 [1.90975259 0.23854512]
 [0.82101674 1.486927  ]
 [2.67711679 3.2458653 ]
 [2.11629267 2.07348825]
 [2.98415193 1.61628653]
 [1.09072993 2.74396034]
 [1.41454994 3.66498748]
 [0.25932556 3.12906183]
 [1.57057144 1.04490049]
 [2.33630388 3.18627512]
 [2.33026504 3.18249138]
 [2.32988246 3.17866835]
 [2.32957635 3.17986795]
 [2.32953349 3.17865498]]
Outputs = [[-8.56031735e-01 -2.49278090e+00 -8.09384637e+00]
 [-2.14829771e+00 -1.82086408e+00 -3.69656346e+00]
 [-2.30794374e+00 -2.38698546e+00  8.78522258e-01]
 [-5.92298209e+00 -5.32604759e+00  2.07292062e+00]
 [-4.18978092e+00 -4.76513455e-02 -1.81905146e+00]
 [-4.60043845e+00 -1.76339935e+01  1.61233138e+00]
 [-3.83469027e+00 -1.22324751e+00  2.62392865e+00]
 [-5.07953742e+00  2.93328109e-01  1.93708654e+00]
 [-3.38838739e+00  7.21535433e-01 -1.33536857e+01]
 [-2.61547192e+00 -1.86486036e+

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [9]:
xlimits = [[0.0, 25.0]]
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.solver.solver_impl:Train surrogates with 3 points...
INFO:egobox_ego.solver.solver_impl:Objective model hyperparameters optim init >>> [Partial { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, active: [0] }]
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox_ego.solver.solver_impl:... Objective trained (1 / Mixture[Hard])
INFO:egobox_ego.solver.solver_computations:Infill criterion WB2 scaling is updated to 13.123914263152923
INFO:egobox_ego.solver.solver_infill_optim:Optimize infill criterion...
INFO:egobox_ego.solver.solver_impl:Train surrogates with 4 points...
INFO:egobox_ego.solver.solver_impl:Objective model hyperparameters optim init >>> [Partial { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, active: [0] }]
INFO:egobox_ego.so

Optimization f=[-15.12510314] at [18.93554901]


## Example 2 : Mixed-integer optimization

### Test function

In [10]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [11]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 3, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, gp: GpConfig { regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, recombination: Hard, theta_tuning: Full { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }, n_start: 10, max_eval: 50 }, infill_criterion: EI, infill_optimizer: Cobyla, target: -15.12, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Int(0, 25)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: MeanConstraint }
INFO:egobox_ego.

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[21.]
 [10.]
 [ 1.]
 [24.]
 [20.]
 [18.]
 [19.]]
Outputs = [[-11.44370682]
 [  5.70983099]
 [  1.78601478]
 [  4.91604976]
 [-14.15453288]
 [-14.43198471]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [12]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [13]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 10, n_start: 20, n_doe: 0, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, gp: GpConfig { regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, recombination: Hard, theta_tuning: Full { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.01, 10.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }, n_start: 10, max_eval: 50 }, infill_criterion: WB2, infill_optimizer: Cobyla, target: -inf, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Float(-5.0, 5.0), Enum(3), Enum(2), Ord([0.0, 2.0, 3.0])], seed: Some(42), trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false,

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[ 0.69939824  0.          0.          0.        ]
 [ 4.84411847  1.          0.          0.        ]
 [-4.75038813  1.          0.          2.        ]
 [-1.81967258  2.          1.          2.        ]
 [ 2.46052467  0.          0.          2.        ]
 [-2.82859054  0.          0.          2.        ]
 [ 2.5012666   2.          1.          0.        ]
 [-0.6935668   2.          1.          3.        ]
 [-4.91686045  2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          2.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [14]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(xspecs, gp_config=Ellipsis, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, infill_strategy=Ellipsis, cstr_infill=False, cstr_strategy=Ellipsis, q_points=1, q_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, trego=False, coego_n_coop=0, q_optmod=1, target=Ellipsis, outdir=None, warm_start=False, hot_start=None, seed=None)
 |  
 |  Optimizer constructor
 |  xspecs (list(XSpec)) where XSpec(xtype=FLOAT|INT|ORD|ENUM, xlimits=[<f(xtype)>] or tags=[strings]):
 |      Specifications of the nx components of the input x (eg. len(xspecs) == nx)
 |      Depending on the x type we get the following for xlimits:
 |      * when FLOAT: xlimits is [float lower_bound, float upper_bound],
 |      * when INT: xlimits is [int lower_bound, int upper_bound],
 |      * when ORD: xlimits is [float_1, float_2, ..., float_n],
 |      * when ENUM: xlimits is just the int size of the enumeration otherwise a list of tags is specified

In [15]:
help(egx.GpConfig)

Help on class GpConfig in module builtins:

class GpConfig(object)
 |  GpConfig(regr_spec=Ellipsis, corr_spec=Ellipsis, kpls_dim=Ellipsis, n_clusters=Ellipsis, recombination=Ellipsis, theta_init=Ellipsis, theta_bounds=Ellipsis, n_start=Ellipsis, max_eval=Ellipsis)
 |  
 |  GP configuration used by `Egor` and `GpMix`
 |  
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  corr_spec
 |      (CorrelationSpec flags, an int in [1, 15])
 |      Specification of correlation models used in mixture.
 |      Can be CorrelationSpec.SQUARED_EXPONENTIAL (1), CorrelationSpec.ABSOLUTE_EXPONENTIAL (2),
 |      CorrelationSpec.MATERN32 (4), CorrelationSpec.MATERN52 (8) or
 |      any bit-wise union of these values (e.g. CorrelationSpec.MATERN32 | CorrelationSpec.MATER